In [1]:
%pip install pymilvus
%pip install pymilvus[model]
%pip install numpy
%pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

# 1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=False,
)

# 2. Add fields to schema
schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="author", datatype=DataType.VARCHAR, max_length=25)

# 3. Prepare index parameters
index_params = client.prepare_index_params()

# 4. Add indexes
# index_params.add_index(
#     field_name="my_id",
#     index_type="STL_SORT"
# )

# index_params.add_index(
#     field_name="my_vector", 
#     index_type="AUTOINDEX",
#     metric_type="L2",
#     params={"nlist": 1024}
# )

# 5. Create a collection
client.create_collection(
    collection_name="customized_setup1",
    schema=schema,
)

In [19]:

res = client.insert(
    collection_name="customized_setup",
    data=[
        {
            'my_id': 0,
            'my_vector': [
                0.27417449965222693,
                -0.9401784221711342,
                -0.8344432775467099,
                0.9797361846081416,
                0.6294256393761057
            ],
            'author': 'John Doe'
        },
        {
            'my_id': 1,
            'my_vector': [
                0.57417449965222693,
                -0.9401784221711342,
                0.9197526367693833,
                0.49519396415367245,
                -0.558567588166478
            ],
            'author': 'Jane Doe'
        },
        {
            'my_id': 2,
            'my_vector': [
                0.56949086179692356,
                -0.533609076732849,
                -0.8344432775467099,
                0.9797361846081416,
                0.6294256393761057
            ],
            'author': 'John Doe'
        }
    ]
)

# {'upsert_count': 1}

# # 4. Upsert multiple records
# res = client.upsert(
#     collection_name="test_collection",
#     data=[
#         {
#             'id': 1,
#              'vector': [
#                  0.3457690490452393,
#                  -0.9401784221711342,
#                  0.9123948134344333,
#                  0.49519396415367245,
#                  -0.558567588166478
#              ]
#        },
#        {
#            'id': 2,
#            'vector': [
#                0.42349086179692356,
#                -0.533609076732849,
#                -0.8344432775467099,
#                0.675761846081416,
#                0.57094256393761057
#            ]
#        }
#    ]
# )

# {'upsert_count': 2}

In [25]:
# 4. Upsert a record
res = client.insert(
    collection_name="customized_setup1",
    data={
        'my_id': 0,
        'my_vector': [
            0.6186516144460161,
            0.5927442462488592,
            0.848608119657156,
            0.9287046808231654,
            -0.42215796530168403
        ],
        'author': 'John'
    }
)

In [28]:
res = client.upsert(
    collection_name="customized_setup1",
    data={
        'my_id': 0,
        'my_vector': [
            0.6186516144460161,
            0.5927442462488592,
            0.848608119657156,
            0.9287046808231654,
            -0.42215796530168403
        ],
        'author': 'Jesus'
    }
)

In [5]:
client.insert(
    collection_name="test_collection",
    data=[
        {
            'id': 0,
            'vector': [
                0.37417449965222693,
                -0.9401784221711342,
                -0.8344432775467099,
                0.9797361846081416,
                0.6294256393761057
            ]
        },
        {
            'id': 1,
            'vector': [
                0.37417449965222693,
                -0.9401784221711342,
                0.9197526367693833,
                0.49519396415367245,
                -0.558567588166478
            ]
        },
        {
            'id': 2,
            'vector': [
                0.46949086179692356,
                -0.533609076732849,
                -0.8344432775467099,
                0.9797361846081416,
                0.6294256393761057
            ]
        }
    ]
)

{'insert_count': 3, 'ids': [0, 1, 2]}

In [4]:
%pip install pymilvus

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

In [1]:
# hello_sprase.py demonstrates the basic operations of PyMilvus, a Python SDK of Milvus,
# while operating on sparse float vectors.
# 1. connect to Milvus
# 2. create collection
# 3. insert data
# 4. create index
# 5. search, query, and hybrid search on entities
# 6. delete entities by PK
# 7. drop collection
import time

import numpy as np
from scipy.sparse import rand
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

fmt = "=== {:30} ==="
search_latency_fmt = "search latency = {:.4f}s"
num_entities, dim, density = 1000, 3000, 0.005

def log(msg):
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " " + msg)

# -----------------------------------------------------------------------------
# connect to Milvus
log(fmt.format("start connecting to Milvus"))
connections.connect("default", host="localhost", port="19530")

has = utility.has_collection("hello_sparse")
log(f"Does collection hello_sparse exist in Milvus: {has}")

# -----------------------------------------------------------------------------
# create collection with a sparse float vector column
hello_sparse = None
if not has:
    fields = [
        FieldSchema(name="pk", dtype=DataType.VARCHAR, is_primary=True, max_length=100),
        FieldSchema(name="random", dtype=DataType.DOUBLE),
        FieldSchema(name="embeddings", dtype=DataType.SPARSE_FLOAT_VECTOR),
    ]
    schema = CollectionSchema(fields, "hello_sparse is the simplest demo to introduce sparse float vector usage")
    log(fmt.format("Create collection `hello_sparse`"))
    hello_sparse = Collection("hello_sparse", schema, consistency_level="Strong")
else:
    hello_sparse = Collection("hello_sparse")

log(f"hello_sparse has {hello_sparse.num_entities} entities({hello_sparse.num_entities/1000000}M), indexed {hello_sparse.has_index()}")

2024-06-07 14:00:45 === start connecting to Milvus     ===


KeyboardInterrupt: 

In [43]:
# create index
if not hello_sparse.has_index():
    log(fmt.format("Start Creating index SPARSE_INVERTED_INDEX"))
    index = {
        "index_type": "SPARSE_INVERTED_INDEX",
        "metric_type": "IP",
        "params":{
            "drop_ratio_build": 0.2,
        }
    }
    hello_sparse.create_index("embeddings", index)

log(fmt.format("Start loading"))
hello_sparse.load()

# -----------------------------------------------------------------------------

2024-05-03 20:02:09 === Start loading                  ===


In [83]:
import random
rng = np.random.default_rng()

# insert
log(fmt.format("Start creating entities to insert"))
rng = np.random.default_rng(seed=19530)

matrix_csr = rand(num_entities, dim, density=density, format='csr')
entities = [
    ["test_pos"],
    rng.random(1).tolist(),
    rand(num_entities, dim, density=density, format='csr')
]

#entities
insert_result = hello_sparse.insert(data=entities)

# print(f"insert_result.primary_keys: {insert_result.primary_keys}")

2024-05-03 20:32:17 === Start creating entities to insert ===


RPC error: [batch_insert], <ParamError: (code=1, message=('Field data size misaligned for field [embeddings] ', 'got size=[1000] ', 'alignment size=[1]'))>, <Time:{'RPC start': '2024-05-03 20:32:18.364464', 'RPC error': '2024-05-03 20:32:18.365421'}>


ParamError: <ParamError: (code=1, message=('Field data size misaligned for field [embeddings] ', 'got size=[1000] ', 'alignment size=[1]'))>

In [28]:
# -----------------------------------------------------------------------------
# # insert
# log(fmt.format("Start creating entities to insert"))
# rng = np.random.default_rng(seed=19530)
# # this step is so damn slow
matrix_csr = rand(num_entities, dim, density=density, format='csr')
# entities = [
#     rng.random(num_entities).tolist(),
#     matrix_csr,
# ]

# log(fmt.format("Start inserting entities"))
# insert_result = hello_sparse.insert(entities)

# print(f"insert_result.primary_keys: {insert_result.primary_keys}")

#insert_result contains the primary keys, upsert different entities with the same primary key will update the entity
#test  = [{'pk': pk, 'embedings': matrix_csr} for pk in insert_result.primary_keys] #'random': rng.random(num_entities).tolist(), 
hello_sparse.upsert(data=[{"pk": "449513279599944426", "random": rng.random(num_entities).tolist(), "embeddings": matrix_csr}])
test
# -----------------------------------------------------------------------------

RPC error: [upsert_rows], <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, please check it.)>, <Time:{'RPC start': '2024-05-03 19:51:28.681218', 'RPC error': '2024-05-03 19:51:28.681768'}>


DataNotMatchException: <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, please check it.)>

In [ ]:
# search based on vector similarity
log(fmt.format("Start searching based on vector similarity"))
vectors_to_search = entities[-1][-1:]
search_params = {
    "metric_type": "IP",
    "params": {
        "drop_ratio_search": "0.2",
    }
}

start_time = time.time()
result = hello_sparse.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["pk", "random", "embeddings"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}")
log(search_latency_fmt.format(end_time - start_time))
# -----------------------------------------------------------------------------

In [ ]:
# query based on scalar filtering(boolean, int, etc.)
print(fmt.format("Start querying with `random > 0.5`"))

start_time = time.time()
result = hello_sparse.query(expr="random > 0.5", output_fields=["random", "embeddings"])
end_time = time.time()

print(f"query result:\n-{result[0]}")
print(search_latency_fmt.format(end_time - start_time))

# -----------------------------------------------------------------------------
# pagination
r1 = hello_sparse.query(expr="random > 0.5", limit=4, output_fields=["random"])
r2 = hello_sparse.query(expr="random > 0.5", offset=1, limit=3, output_fields=["random"])
print(f"query pagination(limit=4):\n\t{r1}")
print(f"query pagination(offset=1, limit=3):\n\t{r2}")


# -----------------------------------------------------------------------------

In [ ]:
# hybrid search
print(fmt.format("Start hybrid searching with `random > 0.5`"))

start_time = time.time()
result = hello_sparse.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > 0.5", output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))

In [21]:
# -----------------------------------------------------------------------------
# delete entities by PK
# You can delete entities by their PK values using boolean expressions.
ids = insert_result.primary_keys

expr = f'pk in ["{ids[0]}" , "{ids[1]}"]'
print(fmt.format(f"Start deleting with expr `{expr}`"))

result = hello_sparse.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query before delete by expr=`{expr}` -> result: \n-{result[0]}\n-{result[1]}\n")

hello_sparse.delete(expr)

result = hello_sparse.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query after delete by expr=`{expr}` -> result: {result}\n")


# -----------------------------------------------------------------------------
# drop collection
print(fmt.format("Drop collection `hello_sparse`"))
utility.drop_collection("hello_sparse")

RPC error: [query], <MilvusException: (code=101, message=failed to query: collection not loaded[collection=449513279599936303])>, <Time:{'RPC start': '2024-05-03 19:46:26.827462', 'RPC error': '2024-05-03 19:46:26.844953'}>


=== Start deleting with expr `pk in ["449513279599948430" , "449513279599948431"]` ===


MilvusException: <MilvusException: (code=101, message=failed to query: collection not loaded[collection=449513279599936303])>

In [ ]:
# A demo showing hybrid semantic search with dense and sparse vectors using Milvus.
# You can optionally choose to use the BGE-M3 model to embed the text as dense
# and sparse vectors, or simply use random generated vectors as an example.
# Note that the sparse vector search feature is only available in Milvus 2.4.0 or
# higher version. Make sure you follow https://milvus.io/docs/install_standalone-docker.md
# to set up the latest version of Milvus in your local environment.

# To connect to Milvus server, you need the python client library called pymilvus.
# To use BGE-M3 model, you need to install the optional `model` module in pymilvus.
# You can get them by simply running the following commands:
# pip install pymilvus
# pip install pymilvus[model]

# If true, use BGE-M3 model to generate dense and sparse vectors.
# If false, use random numbers to compose dense and sparse vectors.
use_bge_m3 = True

# The overall steps are as follows:
# 1. embed the text as dense and sparse vectors
# 2. setup a Milvus collection to store the dense and sparse vectors
# 3. insert the data to Milvus
# 4. search and inspect the result!

from pymilvus.model import BGEM3EmbeddingFunction
import random
import string
import numpy as np

from pymilvus import (
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection, AnnSearchRequest, RRFRanker, connections,
)

# 1. prepare a small corpus to search
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]
# add some randomly generated texts
docs.extend([' '.join(''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(1, 8))) for _ in range(10)) for _ in range(1000)])
query = "Who started AI research?"

def random_embedding(texts):
    rng = np.random.default_rng()
    return {
        "dense": np.random.rand(len(texts), 768),
        "sparse": [{d: rng.random() for d in random.sample(range(1000), random.randint(20, 30))} for _ in texts],
    }

dense_dim = 768
ef = random_embedding

if use_bge_m3:
    # BGE-M3 model can embed texts as dense and sparse vectors.
    # It is included in the optional `model` module in pymilvus, to install it,
    # simply run "pip install pymilvus[model]".
    from pymilvus.model.hybrid import BGEM3EmbeddingFunction
    ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
    dense_dim = ef.dim["dense"]

docs_embeddings = ef(docs)
query_embeddings = ef([query])

# 2. setup Milvus collection and index
connections.connect("default", host="localhost", port="19530")

# Specify the data schema for the new Collection.
fields = [
    # Use auto generated id as primary key
    FieldSchema(name="pk", dtype=DataType.VARCHAR,
                is_primary=True, auto_id=True, max_length=100),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    # Milvus now supports both sparse and dense vectors, we can store each in
    # a separate field to conduct hybrid search on both vectors.
    FieldSchema(name="text_sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="text_dense_vector", dtype=DataType.FLOAT_VECTOR,
                dim=dense_dim),
]
schema = CollectionSchema(fields, "")
col_name = 'hybrid_demo'
# Now we can create the new collection with above name and schema.
col = Collection(col_name, schema, consistency_level="Strong")

# We need to create indices for the vector fields. The indices will be loaded
# into memory for efficient search.
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("text_sparse_vector", sparse_index)
dense_index = {"index_type": "FLAT", "metric_type": "L2"}
col.create_index("text_dense_vector", dense_index)
col.load()

# 3. insert text and sparse/dense vector representations into the collection
entities = [docs, docs_embeddings["sparse"], docs_embeddings["dense"]]
col.insert(entities)
col.flush()

# 4. search and inspect the result!
k = 2 # we want to get the top 2 docs closest to the query

# Prepare the search requests for both vector fields
sparse_search_params = {"metric_type": "IP"}
sparse_req = AnnSearchRequest(query_embeddings["sparse"],
                              "text_sparse_vector", sparse_search_params, limit=k)
dense_search_params = {"metric_type": "L2"}
dense_req = AnnSearchRequest(query_embeddings["dense"],
                             "text_dense_vector", dense_search_params, limit=k)

# Search topK docs based on dense and sparse vectors and rerank with RRF.
res = col.hybrid_search([sparse_req, dense_req], rerank=RRFRanker(),
                        limit=k, output_fields=['text'])

# Currently Milvus only support 1 query in the same hybrid search request, so
# we inspect res[0] directly. In future release Milvus will accept batch
# hybrid search queries in the same call.
for hit in res[0]:
    print(f'text: {hit.fields["text"]} distance {hit.distance}')

# If you are using BGE-M3 to generate the embedding, you should see the following:
# text: Alan Turing was the first person to conduct substantial research in AI. distance 0.032786883413791656
# text: Artificial intelligence was founded as an academic discipline in 1956. distance 0.016129031777381897

# Drop the collection to clean up the data.
utility.drop_collection(col_name)